In [1]:
import ast

import numpy as np
import pandas as pd
import pysgpp as pysgpp

from pysgpp.extensions.datadriven.learner import Types
from pysgpp.extensions.datadriven.learner import LearnerBuilder
from pysgpp.extensions.datadriven.uq.plot.plot1d import plotSG1d
from pysgpp.extensions.datadriven.uq.plot.plot2d import plotSG2d,plotGrid2d
from pysgpp.extensions.datadriven.uq.plot.plot3d import plotSG3d
from pysgpp.extensions.datadriven.uq.plot.plotGrid import plotGrid

import matplotlib.pyplot as plt
from sklearn import preprocessing
plt.style.use('ggplot')
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 12)

Populating the interactive namespace from numpy and matplotlib


In [2]:
#filename = 'output_100_configs.csv'
filename = '2D_param_space_times.csv'


df = pd.read_csv(filename, header = None, names = ['params', 'time'])
df.params = df.params.apply(ast.literal_eval)

for param in df.iloc[0].params.keys():
    df[param] = df.params.apply(lambda row: row[param])
    
df['density'] = (df.MOL_X*df.MOL_Y)/(df.DOMAIN_SIZE_X*df.DOMAIN_SIZE_Y)
del df['params']
df = df[df.time > 0]

In [3]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [4]:
columns = ['DOMAIN_SIZE_X', 'DOMAIN_SIZE_Y', 'MOL_X', 'MOL_Y', 'LINKED_CELL_SIZE_X', 'LINKED_CELL_SIZE_Y',
          'BLOCK_SIZE', 'CUTOFF_RADIUS', 'density','time']

In [22]:
params_ = ['CUTOFF_RADIUS', 'density', 'MOL_Y','MOL_X','DOMAIN_SIZE_Y'
          ,'DOMAIN_SIZE_X','MOL_X','LINKED_CELL_SIZE_X', 'LINKED_CELL_SIZE_Y',]

In [23]:
params_2 = ['density','LINKED_CELL_SIZE_Y']

In [24]:
params_4 = ['density','CUTOFF_RADIUS']

In [25]:
normalized_df = normalize(df[columns])
num_samples = normalized_df.shape[0]

In [26]:
X = normalized_df[params_].values
Y = normalized_df['time'].values

In [27]:
numDims = X.shape[1]
print X.shape

(100, 9)


In [28]:
for levels in range(1, 4):
    builder = LearnerBuilder()
    builder.buildRegressor()
    builder.withTrainingDataFromNumPyArray(X, Y)
    builder = builder.withGrid().withBorder(Types.BorderTypes.NONE)
    builder.withLevel(levels)
    builder = builder.withSpecification()
    builder.withLambda(0.0000001).withAdaptPoints(100)

    builder.withIdentityOperator()
    builder = builder.withStopPolicy()
    builder = builder.withCGSolver()
    builder.withAccuracy(0.0001)
    builder.withImax(1000)
    learner = builder.andGetResult()

    learner.learnData()
    print 'Level: %i' % levels
    print 'L2Norm: %f' % learner.getL2NormError()
    print 'MaxError: %f' % learner.getMaxError()
    print 'MinError: %f' % learner.getMinError()
    print 'Error: %f' % learner.evalError(learner.dataContainer,learner.alpha)
    print 'RMSerror: %f' % learner.error.RMSNorm()
    print 'Grid size BEFORE refinement: %i' % learner.grid.getSize()
    print 'Refining grid.. '
    learner.refineGrid()
    learner.learnData()
    print 'Grid size AFTER refinement: %i' % learner.grid.getSize()
    print 'L2Norm: %f' % learner.getL2NormError()
    print 'MaxError: %f' % learner.getMaxError()
    print 'MinError: %f' % learner.getMinError()
    print 'Error: %f' % learner.evalError(learner.dataContainer,learner.alpha)
    print 'RMSerror: %f' % learner.error.RMSNorm()
    print '--------------------------------------------'

Level: 1
L2Norm: 2.259309
MaxError: 0.759811
MinError: 0.000015
Error: 0.051045
RMSerror: 0.109189
Grid size BEFORE refinement: 1
Refining grid.. 
Grid size AFTER refinement: 19
L2Norm: 0.538665
MaxError: 0.198511
MinError: 0.000203
Error: 0.002902
RMSerror: 0.006640
--------------------------------------------
Level: 2
L2Norm: 0.538665
MaxError: 0.198511
MinError: 0.000203
Error: 0.002902
RMSerror: 0.006640
Grid size BEFORE refinement: 19
Refining grid.. 
Grid size AFTER refinement: 199
L2Norm: 0.315026
MaxError: 0.121987
MinError: 0.000271
Error: 0.000992
RMSerror: 0.002279
--------------------------------------------
Level: 3
L2Norm: 0.315023
MaxError: 0.121841
MinError: 0.000225
Error: 0.000992
RMSerror: 0.002277
Grid size BEFORE refinement: 199
Refining grid.. 
Grid size AFTER refinement: 1310
L2Norm: 0.016127
MaxError: 0.005687
MinError: 0.000005
Error: 0.000003
RMSerror: 0.000006
--------------------------------------------
